In [2]:
from langchain.llms import HuggingFacePipeline



In [ ]:
MODEL_NAME = "Yukang/Llama-2-13b-chat-longlora-32k-sft"

llm = HuggingFacePipeline.from_model_id(
    model_id=MODEL_NAME,
    task="text-generation",
    model_kwargs={"temperature": 0, "max_length": 64},
)



/Users/karthiksoman/anaconda3/envs/llm_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/karthiksoman/anaconda3/envs/llm_env/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
Loading checkpoint shards:   0%|                          | 0/3 [00:00<?, ?it/s]